In [90]:
### WIP



In [91]:
# https://github.com/SeldonIO/alibi-detect#drift-detection

In [81]:
!pip install alibi-detect
!pip install tensorflow
!pip install alibi

In [82]:
import pandas as pd
import numpy as np

import alibi
import matplotlib.pyplot as plt

from alibi_detect.cd import ChiSquareDrift, TabularDrift
from alibi_detect.utils.saving import save_detector, load_detector

In [83]:
data = pd.read_csv('../../data.csv')

# cd = MMDDrift(x_ref, backend='tensorflow', p_val=.05)
# preds = cd.predict(x)

In [84]:
data.head(5)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,148,Male,0,No,No,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,45.65,45.65,Yes
1,463,Male,0,Yes,Yes,4,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,No,Electronic check,101.15,385.90,Yes
2,471,Female,1,No,No,17,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,20.65,330.60,No
3,496,Male,0,No,No,22,No,No phone service,DSL,No,...,Yes,No,No,Yes,One year,Yes,Bank transfer (automatic),43.75,903.60,Yes
4,833,Female,0,Yes,Yes,70,Yes,No,DSL,Yes,...,Yes,Yes,No,Yes,One year,No,Credit card (automatic),74.10,5222.30,No


In [85]:
# do all data massaging
# Convert binary variable into numeric so plotting is easier. We need to later take mean
data['Churn'] = data['Churn'].map({'Yes': 1, 'No': 0})

data.replace(" ", np.nan, inplace=True)

data['TotalCharges'] = pd.to_numeric(data['TotalCharges'])

mean = data['TotalCharges'].mean()
data.fillna(mean, inplace=True)
data_enc = data.drop(['Churn', 'customerID'], axis=1)
data_enc.head(5)
labels = data['Churn']

# import category_encoders as ce
# import joblib

# names = ['gender', 'Partner', 'Dependents', 'PhoneService', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']
# # for column in names:
# #     labelencoder(column)
# data_enc = data
# data_enc = data_enc.drop(['Churn', 'customerID'], axis=1)
# enc = ce.ordinal.OrdinalEncoder(cols=names)
# enc.fit(data_enc)
# labelled_set = enc.transform(data_enc)

categories_per_feature = ['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod', 'OnlineSecurity', 'OnlineBackup',
         'DeviceProtection', 'TechSupport']

# ohe = ce.OneHotEncoder(cols=names)
# data_ohe = data
# data_ohe = data_ohe.drop(['Churn', 'customerID'], axis=1)
# ohe.fit(data_ohe)
# final_set = ohe.transform(labelled_set)

# final_set.head(5)

In [86]:

from sklearn.model_selection import train_test_split
labels = data['Churn']
X_train, X_test, y_train, y_test = train_test_split(data_enc, labels, test_size=0.3)
print ('Training Data Shape',X_train.shape, y_train.shape)
print ('Testing Data Shape',X_test.shape, y_test.shape)

Y = data['Churn']
X = data_enc

X_test_1, X_test_2, y_test_1, y_test_2 = train_test_split(X_test, y_test, test_size=0.5)



Training Data Shape (4930, 19) (4930,)
Testing Data Shape (2113, 19) (2113,)


In [87]:
print(X_train.shape[1])


print(X_train.shape)

19
(4930, 19)


In [88]:
# from alibi_detect.od import IForest
# od = IForest(threshold=0.05, n_estimators=500)

# od.fit(X_train.values)
X_train.shape
cd = TabularDrift(X_train.values, p_val=.05, categories_per_feature=categories_per_feature)

In [89]:

# od_preds = od.predict(X_test, return_instance_score=True)
# print(od_preds)
#
#col #34 is the monthly bill amount
X_test_1['tenure']=40
X_test_1['MonthlyCharges']=40
X_test_1['TotalCharges']=7000

input_with_insane_monthly_charges = X_test_1.iloc[2]
# input_with_insane_monthly_charges.values[34] = 22975.0
# input_with_insane_monthly_charges.values[35] = 25000.0
print(input_with_insane_monthly_charges.values.reshape(1,19))

cd_preds = cd.predict(input_with_insane_monthly_charges.values.reshape(1,19))
print(cd_preds)

# from alibi_detect.cd import MMDDrift

# cd1 = MMDDrift(X_train.values, backend='tensorflow', p_val=.05)
# preds = cd1.predict(input_with_insane_monthly_charges.values.reshape(1,19))
# print(preds)


[['Female' 0 'No' 'No' 40 'Yes' 'No' 'Fiber optic' 'No' 'Yes' 'No' 'No'
  'Yes' 'Yes' 'One year' 'Yes' 'Mailed check' 40 7000]]
{'data': {'is_drift': 0, 'distance': array([0.5026369 , 0.1653144 , 0.48133874, 0.30223125, 0.59614605,
       0.09716024, 0.5227181 , 0.34523326, 0.50020283, 0.6596349 ,
       0.55638945, 0.49979717, 0.6162272 , 0.60993916, 0.54969573,
       0.4117647 , 0.77728194, 0.74016225, 0.953144  ], dtype=float32), 'p_val': array([0.9947262 , 1.        , 1.        , 1.        , 0.8077079 ,
       1.        , 0.9545639 , 1.        , 0.99959433, 0.6807302 ,
       0.8872211 , 1.        , 0.76754564, 0.7801217 , 0.90060854,
       1.        , 0.44543612, 0.51967543, 0.09371196], dtype=float32), 'threshold': 0.002631578947368421}, 'meta': {'name': 'TabularDrift', 'detector_type': 'offline', 'data_type': None}}



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
